# Assignment

Using matplotlib/pyplot, implement a function that receives a pandas dataframe with three columns (chromosome, position, pvalue) and draws a manhattan plot with a result similar to this:

![manhattan](https://github.com/ne1s0n/dataviz_python/raw/main/resources/proj01_manhattan_target.png)





# Data

It is provided a GWAS result dataset originally included with the [Plink](https://github.com/ShujiaHuang/qmplot/blob/main/tests/data/gwas_plink_result.tsv) software, which I saved in the course repo for extra security. The file is:

* tab separated
* without header
* 9224 rows (one per SNP)
* 12 columns:
  * `"CHROM", "POS", "ID", "REF", "ALT", "A1", "TEST", "OBS_CT", "BETA", "SE", "T_STAT", "P"`
* some rows have a missing value in the `P` column, and need to be removed (see [.dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna))

You may need a refresh on the [pandas.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

The dataset is available at the following address:


In [ ]:
GWAS_RESULTS_URL = 'https://github.com/ne1s0n/dataviz_python/raw/main/resources/gwas_plink_result_cleaned.tsv'

# Is there anybody NOT familiar with manhattan plots?

Raise your hand!

# Existing solutions

Many software solutions are already available to create manhattan plots with more options that we will ever able to implement. There's no doubt that if you *need* to do a manhattan plot it's better to not reinvent the wheel and use something stable and published. The whole point of this project is to do meaningful exercise, though, not to create a competitive tool. 

That said, you may want to take a look to:

* [manhattan plots with bioinfokit](https://www.reneshbedre.com/blog/manhattan-plot.html), a bioinformatics-oriented package
* [manhattan plots with plotly](https://plotly.com/python/manhattan-plot/), a generic visualization library with emphasis on interactivity
* [manhattan plots with qmplot](https://pythonawesome.com/a-python-package-for-creating-high-quality-manhattan-and-q-q-plots-from-gwas-results/) : a dedicated package for creating high-quality manhattan and Q-Q plots from GWAS results 

# That's it

That's the assignment. Keep scrolling if you want to read and follow the subassignments (SA).

<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>
<br><br><br><br><br><br>

# SA0) Read the data

Read the data from the remote url saved in `GWAS_RESULTS_URL` variable, remembering that there's no file header but you should name the columns

`"CHROM", "POS", "ID", "REF", "ALT", "A1", "TEST", "OBS_CT", "BETA", "SE", "T_STAT", "P"`

Moreover remember to

* remove lines with missing (`NA`) P values
* compute -log10(p)

In [ ]:
import pandas as pd
import numpy as np

colnames = ["CHROM", "POS", "ID", "REF", "ALT", "A1", "TEST", "OBS_CT", "BETA", "SE", "T_STAT", "P"]
df = pd.read_csv(filepath_or_buffer = GWAS_RESULTS_URL, sep = '\t', names = colnames)
print(df.shape)
df.dropna(inplace = True)
print(df.shape)

#computing the -log10(p)
df['SCORE'] = -np.log10(df['P'])
print(df.shape)


df.head()

# SA1) A simpler problem: single chromosome manhattan plot 

At this point you should already be able to do a single chromosome plot.

In [ ]:
import matplotlib.pyplot as plt

#focusing on chromosome 1
df_ch1 = df[df['CHROM'] == 'chr01']

plt.scatter(x = df_ch1['POS'], y=df_ch1['SCORE'])

# SA2) Sort the data by chromosome

In [ ]:
df.sort_values(by = 'CHROM', inplace = True)

# SA3) Compute the length of each chromosome

This will certainly come handy. Put the values in a variable.



In [ ]:
chroms = df.groupby('CHROM')['POS'].max()
print(chroms)

In [ ]:
chroms['chr01'] * 3

# SA4) Create a new column SHIFT column

For chrom 1, SHIFT should be zero

For chrom 2, SHIFT should be the lenght of chrom 1

For chrom 3, SHIFT should be the lenght of chrom 1 plus the length of chrom 2

And so forth.


In [ ]:
#first trial: let's just print the chromosome codes
for c in chroms.index:
  print(c)

In [ ]:
#an improvement: let's compute the cumulative shift
shift = 0
for c in chroms.index:
  print('for ' + c + ' shift is ' + str(shift))
  shift = shift + chroms[c]

In [ ]:
#the solution: let's compute the cumulative shift AND update the dataframe

df['SHIFT'] = 0

shift = 0
for c in chroms.index:
  print('for ' + c + ' shift is ' + str(shift))

  sel = df['CHROM'] == c
  df.loc[sel, 'SHIFT'] = shift

  shift = shift + chroms[c]


Taking a look to the SHIFT column

In [ ]:
df[df['CHROM'] == 'chr01'].head()

In [ ]:
df[df['CHROM'] == 'chr02'].head()

In [ ]:
df[df['CHROM'] == 'chr03'].head()

Testing that there is a single value for each chromosome

In [ ]:
df.groupby('CHROM')['SHIFT'].max() - df.groupby('CHROM')['SHIFT'].min()

An alternative approach, without loops

In [ ]:
#the cumulative sum of chromosome lengths
chroms_cumulative = chroms.cumsum()

#they need to be shifted of one
chroms_cumulative[1:23] = chroms_cumulative[0:22]

#chrom 1 gets a zero shift
chroms_cumulative['chr01'] = 0

#ready to map the values
df['SHIFT2'] = df['CHROM'].map(chroms_cumulative)

#testing that we get the same results
all(df['SHIFT2'] == df['SHIFT'])

# SA5) Create new column X (POS + SHIFT)

In [ ]:
df['X'] = df['POS'] + df['SHIFT']

df[df['CHROM'] == 'chr02'].head()

# SA6) Do the plot

In [ ]:
plt.scatter(x=df['X'], y=df['SCORE'])

# SA7) Define a chrom-color dictionary

That is, a dictionary where chromosome names are keys and named colors are values. To obtain the same result as in the example shown at the beginning of this document just alternate "red" and "blue".

In [ ]:
TRUE_color = 'red'
FALSE_color = 'blue'

colors = {}
toggle = True

for c in chroms.index:
  if toggle:
    colors[c] = TRUE_color
  else:
    colors[c] = FALSE_color
  
  toggle = not toggle

colors

# SA8) Redo the plot, using the color map defined

You need to specify the color for every row in your dataframe. However we can use the dictonary we just defined above and the handy [.map()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) pandas function to do the job for us.

In [ ]:
plt.scatter(x=df['X'], y=df['SCORE'], c=df['CHROM'].map(colors))

# SA9) Compute, for each chromosome, the median point

In [ ]:
medians = df.groupby('CHROM')['X'].median()
medians

# SA10) Add a tick + label on the x axis in the middle of each chromosome

You should look for a function that manages X ticks. I wonder how it could be called.

In [ ]:
plt.scatter(x=df['X'], y=df['SCORE'], c=df['CHROM'].map(colors))
plt.xticks(ticks = medians, labels = medians.index)

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(x=df['X'], y=df['SCORE'], c=df['CHROM'].map(colors))
plt.xticks(ticks = medians, labels = medians.index, rotation = -90)

# SA11) Put everything in a function

Functions are handy.

In [ ]:
# we expect the dataframe passed in to have three fields: CHROM, POS and SCORE

def my_manhattan(data):
  #a local copy of the data, so we can add columns to
  #the dataframe without changing the original
  data = data.copy()

  #sorting by chromosome
  data.sort_values(by = 'CHROM', inplace = True)

  #length of each chromosome
  lengths = data.groupby('CHROM')['POS'].max()

  #computing the chromosome shift
  data['SHIFT'] = 0
  shift = 0
  for c in lengths.index:
    sel = data['CHROM'] == c
    data.loc[sel, 'SHIFT'] = shift
    shift = shift + lengths[c]
  
  #computing the actual x values
  data['X'] = data['POS'] + data['SHIFT']
  
  #creating a color dictionary of alternating colors
  values = ['red', 'blue'] * int(len(lengths) / 2)
  if len(lengths) > len(values):
    values.append('red')
  colors = dict(zip(lengths.index, values))

  #computing the medians
  medians = data.groupby('CHROM')['X'].median()

  #doing the plot
  plt.figure(figsize=(12, 8))
  plt.scatter(x=data['X'], y=data['SCORE'], c=data['CHROM'].map(colors))
  plt.xticks(ticks = medians, labels = medians.index, rotation = -90)

  return(None)

my_manhattan(df)

# SA12) Put the function in a module

You'll need to do that from YOUR computer.